# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Turn your project folder path into a string

path = r'/Users/peterreadman/Desktop/Python Projects/CareerFoundry/04-2020 Instacart Basket Analysis/'

# Exercise read-through

In [3]:
# import orders_products_merged_busiest.pkl dataframe from the pkl file

ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', '02b Prepared Data', 'orders_products_merged_busiest.pkl'))

In [4]:
df = ords_prods_merge[:1000000]

In [5]:
df.shape

(1000000, 19)

In [6]:
df.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,Unnamed: 0,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Average orders
2,473747,1,3,3,12,21.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Most orders
3,2254736,1,4,4,7,29.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Average orders
4,431534,1,5,4,15,28.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Most orders


Note that my dataframe shape differs to the one in the exercise. There are five additional columns: '_merge', '_price_range_loc', 'busiest_day', 'busiest_days', 'busiest_period_of_day'

# Grouping Data with pandas

In [7]:
df.groupby('product_name')

# Aggregating Data with agg()

In [8]:
# Performing a Single Aggregation
# 1. Split the data into groups based on “department_id.”
# 2. Apply the agg() function to each group to obtain the mean values for the “order_number” column.

df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


There are some aggregations that can be conducted without use of the agg() function. For instance, the command above could be replaced with a command that uses the mean() function to achieve the same results:

In [9]:
# use mean()

df.groupby('department_id')['order_number'].mean()

department_id
4     18.825780
7     17.472355
13    17.993423
14    19.246334
16    19.463012
17    11.294069
19    19.305237
20    17.599636
Name: order_number, dtype: float64

*key difference in syntax between the two methods: when using agg(), put the column you want to aggregate inside the parentheses of the agg() function as an argument. When using mean() (or any other standard aggregation function), simply index the column with square brackets, then follow it with the function you want to use after the dot.*

**Performing Multiple Aggregations**

In [10]:
# All it comes down to is adding more arguments to your code

df.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
4                18.825780   1  99
7                17.472355   1  99
13               17.993423   1  99
14               19.246334   1  99
16               19.463012   1  99
17               11.294069   1  98
19               19.305237   1  99
20               17.599636   1  99

# Aggregating Data with transform()

To create your flag, you’ll need some criteria. You can use the following:

If the maximum orders the user has made is over 40, then the customer will be labeled a “Loyal customer.”
</br>
</br>
If the maximum orders the user has made is over 10 but less than or equal to 40, then the customer will be labeled a “Regular customer.”
</br>
</br>
If the maximum orders the user has made is less than or equal to 10, then the customer will be labeled a “New customer.”

In [11]:
# Create a loyaly flag column in ords_prods_merge

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [12]:
# Change max_rows display in pandas to 'none'

pd.options.display.max_rows = None

In [13]:
ords_prods_merge.head(5)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,Unnamed: 0,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,196,1,0,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Average orders,10
2,473747,1,3,3,12,21.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Most orders,10
3,2254736,1,4,4,7,29.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Average orders,10
4,431534,1,5,4,15,28.0,196,1,1,195,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Most orders,10


# Deriving Columns with loc()

In [14]:
# Set the flags for loyalty
# set 'Loyal customer' as any customer with 'max_order' of more then 40

ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'


In [15]:
# set 'Regular customer' as any customer with 'max_order' between 11 and 40

ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [16]:
# set 'New customer' as any customer with 'max_order' less than or equal to 10

ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [17]:
# Check the frequency 

ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [18]:
# Check

ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(60)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10
